In [15]:
# Dependencies and dataload
# Link to associated webkit
# http://scikit-learn.org/stable/auto_examples/feature_stacker.html#sphx-glr-auto-examples-feature-stacker-py

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

my_data = pd.read_csv('/Users/stevensun/Prometheus/0_data/prometheus.csv')
my_data.shape

(2376, 127)

In [16]:
#drop incomplete rows

my_data = my_data.dropna()
my_data.shape

(690, 127)

In [17]:
# vectorize data

X = my_data.ix[:,0:109]
y = my_data.ix[:,126]

In [18]:
pca = PCA(n_components=2)
selection = SelectKBest(k=1)

In [21]:
X

,SEX,BMI,AGE (normalized 30-80),ANAT_DEGREE_normalized VAR-VAL_-25_+25,RC_WHT,RC_BLCK,RC_NHAW,RC_AMIND,RC_ASIAN,RC_HISPLAT,...,ST_BI_FEM_YN,ST_ILL_TRACT_YN,ST_LCL_YN,ST_LAT_RETIN_YN,ST_IT_BAND_YN,ST_POPLIT_YN,ST_POST_LAT_CAP_YN,ST_REC_SNIP_YN,ST_QUAD_REL_YN,ST_OTHER_YN
0,0,0.238095,0.38,0.58,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.450794,0.76,0.46,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0.260317,0.60,0.44,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0.673016,0.72,0.44,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0.711111,0.86,0.50,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0.120635,0.78,0.70,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,1,0.396825,0.46,0.46,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1,0.600000,0.96,0.74,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
11,0,0.400000,0.62,0.40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0.431746,0.60,0.50,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)

svm = SVC(kernel="linear")

/Users/stevensun/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  6  11  17  27  83  84  85  88  89 106 107 108] are constant.
  UserWarning)
/Users/stevensun/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
# Do grid search over k, n_components and C:

pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
grid_search.fit(X, y)
print(grid_search.best_estimator_)